In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import torch
import glob
import os

from bs4 import BeautifulSoup

data_path = '/content/drive/MyDrive/iam_dataset/'
save_path = "/content/drive/MyDrive/iam_dataset/saved_model/"


if not os.path.exists(save_path + "calibrated_softmax"):
  os.makedirs(save_path + "calibrated_softmax")

In [ ]:
!cp /content/drive/MyDrive/iam_dataset/temperature_scaling.py .

In [ ]:
# toggle the model for which softmax needs to be calibrated
model_node = 1 # options: 1,2,3,4,5,6,7,23 (for model reuse)

In [ ]:
labelled_data = np.load(data_path + 'labelled_data.npy')

print(labelled_data.shape)

(4901, 2)


In [ ]:
# if model_node is 1,2,3,4...
nodes_handled = [1,2,3,4,23]
if model_node in nodes_handled:
  labelled_data_0 = labelled_data[labelled_data[:,1] == '000']

  print(labelled_data_0.shape)

  if model_node == 1 or model_node == 23:
    labelled_data_0 = labelled_data_0[:196]
    print("node 1, class 0 set")
  elif model_node == 2:
    labelled_data_0 = labelled_data_0[196:(196*2)]
    print("node 2, class 0 set")
  elif model_node == 3:
    labelled_data_0 = labelled_data_0[(196*2):(196*3)]
    print("node 3, class 0 set")
  elif model_node == 4:
    labelled_data_0 = labelled_data_0[:202]
    print("node 4, class 0 set")
  else:
    print("model_node not 1,2,3, or 4")

  print(labelled_data_0.shape)

  print(np.unique(labelled_data_0[:,1]))

  remove_000 = labelled_data[labelled_data[:,1]!='000']

  print(np.unique(remove_000[:,1]))
  print(np.unique(remove_000[:,1]).shape)

  y = [remove_000[remove_000[:,1]==k] for k in np.unique(remove_000[:,1])]
  print(len(y))

  labelled_data_1 = []

  if model_node == 1 or model_node == 23:
    for i in y:
      labelled_data_1.append(i[:4])
    print("node 1, class 1 set")
  elif model_node == 2:
    for i in y:
      labelled_data_1.append(i[5:9])
    print("node 2, class 1 set")  
  elif model_node == 3:
    for i in y:
      labelled_data_1.append(i[9:13])
    print("node 3, class 1 set")
  elif model_node == 4:
    for i in range(len(y)):
      if i == 0:
        labelled_data_1.append(y[i][:10])
      else:
        labelled_data_1.append(y[i][:4])
    print("node 4, class 1 set")
  else:
    print("model_node not 1,2,3, or 4")

  print(len(labelled_data_1))



  if model_node == 1 or model_node == 23 or model_node == 2 or model_node == 3:
    labelled_data_1 = np.array(labelled_data_1)
    print(labelled_data_1.shape)
    labelled_data_1=labelled_data_1.reshape(-1,2)
  if model_node == 4:
    labelled_data_1 = np.concatenate(labelled_data_1)

  print(labelled_data_1.shape)
  print(np.unique(labelled_data_1[:,1]))


  labelled_data_new = np.concatenate((labelled_data_0, labelled_data_1))
  np.random.shuffle(labelled_data_new)
  print(labelled_data_new.shape) # mix of sentences from writer id 0 and others, labelled

else:
  print("node not handled in this block")

(693, 2)
node 1, class 0 set
(196, 2)
['000']
['085' '118' '150' '151' '152' '153' '154' '155' '202' '203' '204' '205'
 '206' '207' '208' '209' '247' '248' '273' '274' '315' '332' '333' '334'
 '335' '336' '337' '338' '339' '340' '341' '342' '343' '344' '345' '346'
 '347' '348' '349' '384' '415' '551' '552' '567' '588' '634' '635' '670'
 '671']
(49,)
49
node 1, class 1 set
49
(49, 4, 2)
(196, 2)
['085' '118' '150' '151' '152' '153' '154' '155' '202' '203' '204' '205'
 '206' '207' '208' '209' '247' '248' '273' '274' '315' '332' '333' '334'
 '335' '336' '337' '338' '339' '340' '341' '342' '343' '344' '345' '346'
 '347' '348' '349' '384' '415' '551' '552' '567' '588' '634' '635' '670'
 '671']
(392, 2)


In [ ]:
from random import uniform as rand

def randConstrained(n, M):
    splits = [0] + [rand(0, 1) for _ in range(0,n-1)] + [1]
    splits.sort()
    diffs = [x - splits[i - 1] for i, x in enumerate(splits)][1:]
    result = map(lambda x:x*M, diffs)
    return result

res = randConstrained(49,1.0)
res_list = list(res)
# print(sum(list(res)))
print(len(res_list))
print(sum(res_list))


# HERE - work with labelled_data for any sampling work
probs = randConstrained(49,1.0)
print(probs)
probs = list(probs)
# print(probs)
probs = np.array(probs)
print(probs[0])
# probs = np.insert(probs, 0, 0.5)
print(probs[1])
print(probs.shape)
def sampling(p, n):
  all_1_writers = np.unique(labelled_data[:,1])[np.unique(labelled_data[:,1])!= '000']
  ixs = np.arange(49)
  return np.array([all_1_writers[np.random.choice(ixs, p=p)] for i in range(n)])

49
1.0
0.00166683483940544
0.05850089194631425
(49,)


In [ ]:
if model_node == 5:

  label_array = sampling(probs, 200)

  labels, counts = np.unique(label_array, return_counts=True)


  print(labels, counts)
  print(labels.shape)
  print(sum(counts))

  all_1_data = labelled_data[labelled_data[:,1]!='000']

  # all_085 = all_1_data[all_1_data[:,1]=='085']
  # print(all_085.shape)

  data_list = []
  # data_list.append(all_085)
  for i in range(len(labels)):
    curr = all_1_data[all_1_data[:,1]==labels[i]][:counts[i]]
    data_list.append(curr)

  print(len(data_list))

  out = np.concatenate(data_list)
  print(out.shape)
  print("node 5, class 1 set")
  all_000 = labelled_data[labelled_data[:,1] == '000']
  np.random.shuffle(all_000)
  all_000 = all_000[:200]
  print(all_000.shape)
  print("node 5, class 0 set")
  labelled_data_new = np.concatenate([all_000, out])
  print(labelled_data_new.shape)

else:
  print("model node not handled in this block")


model node not handled in this block


In [ ]:
nodes = [6,7]
if model_node in nodes:
  if model_node == 6:
    label_array = sampling(probs, 400)
    print("node 6, class 1 set")
  elif model_node == 7:
    label_array = sampling(probs, 200)
    print("node 7, class 1 set")
  else:
    print("set model_node to either 6 or 7")

  labels, counts = np.unique(label_array, return_counts=True)

  print(labels, counts)
  print(labels.shape)
  print(sum(counts))

  all_1_data = labelled_data[labelled_data[:,1]!='000']

  # all_085 = all_1_data[all_1_data[:,1]=='085']
  # print(all_085.shape)

  data_list = []
  # data_list.append(all_085)
  for i in range(len(labels)):
    curr = all_1_data[all_1_data[:,1]==labels[i]][:counts[i]]
    data_list.append(curr)

  print(len(data_list))

  # lbl_data = np.array(data_list)
  out = np.concatenate(data_list)
  # lbl_data = lbl_data.flatten()
  # print(lbl_data)
  print(out.shape)
  all_000 = labelled_data[labelled_data[:,1] == '000']
  np.random.shuffle(all_000)

  if model_node == 6:
    all_000 = all_000[:200]
    print("node 6, class 0 set")
  elif model_node == 7:
    all_000 = all_000[:400]
    print("node 7, class 0 set")
  else:
    print("set model_node to either 6 or 7")
    
  print(all_000.shape)


  labelled_data_new = np.concatenate([all_000, out])
  print(labelled_data_new.shape)

else:
  print("model_node not handled in this block")

model_node not handled in this block


In [ ]:
# split before augmenting to ensure the sentences are being split and not the augmented samples
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(labelled_data_new[:,0], labelled_data_new[:,1], test_size=0.33, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.33, random_state=42)
print(X_train.shape, X_test.shape, X_val.shape, y_train.shape, y_test.shape, y_val.shape)

(175,) (130,) (87,) (175,) (130,) (87,)


In [ ]:
from sklearn.utils import shuffle
from PIL import Image
import random

CROP_SIZE = 113
NUM_LABELS = 50

def get_augmented_sample(sample, label, sample_ratio):
    # Get current image details
    img = Image.open(sample)
    img_width = img.size[0]
    img_height = img.size[1]

    # Compute resize dimensions such that aspect ratio is maintained
    height_fac = CROP_SIZE / img_height
    size = (int(img_width * height_fac), CROP_SIZE)

    # Resize image 
    new_img = img.resize((size), Image.ANTIALIAS)
    new_img_width = new_img.size[0]
    new_img_height = new_img.size[1]

    # Generate a random number of crops of size 113x113 from the resized image
    x_coord = list(range(0, new_img_width - CROP_SIZE))
    num_crops = int(len(x_coord) * sample_ratio)
    random_x_coord = random.sample(x_coord, num_crops)
    
    # Create augmented images (cropped forms) and map them to a label (writer)
    images = []
    labels = []
    for x in random_x_coord:
        img_crop = new_img.crop((x, 0, x + CROP_SIZE, CROP_SIZE))
        # Transform image to an array of numbers
        images.append(np.asarray(img_crop))
        labels.append(label)

    return images, labels

In [ ]:
def augment(data, labels):
    augmented_sample_list = []
    augmented_label_list = []
    for i in range(len(data)):
        augmented_samples, augmented_labels = get_augmented_sample(data[i], labels[i],0.1)
        augmented_sample_list.append(augmented_samples)
        augmented_label_list.append(augmented_labels)
    return augmented_sample_list, augmented_label_list

In [ ]:
augsamps, auglbls = augment(X_train, y_train) # returns two lists of arrays
augsamps_test, auglbls_test = augment(X_test, y_test)
augsamps_val, auglbls_val = augment(X_val, y_val)

In [ ]:
import operator
from functools import reduce

# flattening the lists
aug_samps = reduce(operator.add, augsamps)
aug_lbls = reduce(operator.add, auglbls)

aug_samps_test = reduce(operator.add, augsamps_test)
aug_lbls_test = reduce(operator.add, auglbls_test)

aug_samps_val = reduce(operator.add, augsamps_val)
aug_lbls_val = reduce(operator.add, auglbls_val)

print(len(aug_samps))
print(len(aug_lbls))

print(len(aug_samps_test))
print(len(aug_lbls_test))

print(len(aug_samps_val))
print(len(aug_lbls_val))

26085
26085
19412
19412
14485
14485


In [ ]:
# converting lists to arrays
aug_samps_arr = np.array(aug_samps)
aug_lbls_arr = np.array(aug_lbls)
aug_samps_arr_test = np.array(aug_samps_test)
aug_lbls_arr_test = np.array(aug_lbls_test)
aug_samps_arr_val = np.array(aug_samps_val)
aug_lbls_arr_val = np.array(aug_lbls_val)

print(aug_samps_arr.shape)
print(aug_lbls_arr.shape)
print(aug_samps_arr_test.shape)
print(aug_lbls_arr_test.shape)
print(aug_samps_arr_val.shape)
print(aug_lbls_arr_val.shape)

(26085, 113, 113)
(26085,)
(19412, 113, 113)
(19412,)
(14485, 113, 113)
(14485,)


In [ ]:
X_train = aug_samps_arr
X_train = X_train.reshape(X_train.shape[0], 1, CROP_SIZE, CROP_SIZE)
X_train = X_train.astype('float16')
X_train /= 255.0
y_train = aug_lbls_arr


X_test = aug_samps_arr_test
X_test = X_test.reshape(X_test.shape[0], 1, CROP_SIZE, CROP_SIZE)
X_test = X_test.astype('float16')
X_test /= 255.0
y_test = aug_lbls_arr_test

X_val = aug_samps_arr_val
X_val = X_val.reshape(X_val.shape[0], 1, CROP_SIZE, CROP_SIZE)
X_val = X_val.astype('float16')
X_val /= 255.0
y_val = aug_lbls_arr_val



In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_val.shape)
print(y_val.shape)

(26085, 1, 113, 113)
(26085,)
(19412, 1, 113, 113)
(19412,)
(14485, 1, 113, 113)
(14485,)


In [ ]:
new = np.unique(y_train)
print(new)

y_train[y_train !='000'] = 1
new = np.unique(y_train)
print(new)

y_train[y_train=='000'] = 0
new = np.unique(y_train)
print(new)

print(y_train.shape)

y_train = y_train.astype('float16')

print(np.unique(y_train))

['000' '085' '118' '150' '151' '152' '153' '154' '155' '202' '203' '204'
 '205' '206' '207' '209' '247' '248' '273' '274' '315' '332' '333' '336'
 '337' '338' '339' '340' '341' '342' '343' '344' '345' '348' '349' '415'
 '551' '552' '567' '588' '634' '635' '670' '671']
['000' '1']
['0' '1']
(26085,)
[0. 1.]


In [ ]:
new = np.unique(y_test)
print(new)

y_test[y_test !='000'] = 1
new = np.unique(y_test)
print(new)

y_test[y_test=='000'] = 0
new = np.unique(y_test)
print(new)

print(y_test.shape)

y_test = y_test.astype('float16')

print(np.unique(y_test))

['000' '085' '118' '150' '151' '152' '153' '154' '155' '202' '207' '208'
 '209' '247' '273' '315' '332' '333' '334' '335' '336' '337' '338' '341'
 '342' '343' '344' '345' '346' '347' '349' '384' '415' '551' '552' '567'
 '634' '635' '670' '671']
['000' '1']
['0' '1']
(19412,)
[0. 1.]


In [ ]:
new = np.unique(y_val)
print(new)

y_val[y_val !='000'] = 1
new = np.unique(y_val)
print(new)

y_val[y_val=='000'] = 0
new = np.unique(y_val)
print(new)

print(y_val.shape)

y_val = y_val.astype('float16')

print(np.unique(y_val))

['000' '085' '150' '152' '154' '155' '202' '203' '204' '205' '248' '332'
 '334' '337' '338' '339' '340' '343' '344' '345' '346' '347' '349' '384'
 '551' '552' '567' '588' '634' '635' '670']
['000' '1']
['0' '1']
(14485,)
[0. 1.]


In [ ]:
torch.manual_seed(42)
CUDA = torch.cuda.is_available()
device = torch.device('cuda' if CUDA else 'cpu')
kwargs = {'num_workers': 1, 'pin_memory': True} if CUDA else {}
device

device(type='cuda')

In [ ]:
from torch.utils.data import TensorDataset, DataLoader


tensor_x_train = torch.Tensor(X_train).type(torch.cuda.FloatTensor) # transform to torch tensor
tensor_y_train = torch.Tensor(y_train).type(torch.cuda.FloatTensor).long()



# tensor_x_train = torch.Tensor(X_train) # transform to torch tensor
# tensor_y_train = torch.Tensor(y_train).long()

train_dataset = TensorDataset(tensor_x_train,tensor_y_train) # create your datset
train_dataloader = DataLoader(train_dataset,batch_size=16, shuffle=True, drop_last=True) # create your dataloader

In [ ]:
tensor_x_val = torch.Tensor(X_val).type(torch.cuda.FloatTensor) # transform to torch tensor
tensor_y_val = torch.Tensor(y_val).type(torch.cuda.FloatTensor).long()
# tensor_x_train = torch.Tensor(X_train) # transform to torch tensor
# tensor_y_train = torch.Tensor(y_train).long()

val_dataset = TensorDataset(tensor_x_val,tensor_y_val) # create your datset
val_dataloader = DataLoader(val_dataset,batch_size=16, shuffle=True, drop_last=True) # create your dataloader

In [ ]:
tensor_x_test = torch.Tensor(X_test).type(torch.cuda.FloatTensor) # transform to torch tensor
tensor_y_test = torch.Tensor(y_test).long().type(torch.cuda.FloatTensor).long()

# tensor_x_test = torch.Tensor(X_test) # transform to torch tensor
# tensor_y_test = torch.Tensor(y_test).long()

test_dataset = TensorDataset(tensor_x_test,tensor_y_test) # create your datset
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, drop_last=True) # create your dataloader

In [ ]:
tensor_x_train.shape

torch.Size([26085, 1, 113, 113])

In [ ]:
import torch
from torch import nn
class Binary_Classifier(nn.Module): 
    def __init__(self):
        super(Binary_Classifier, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=3)
        self.conv_drop = nn.Dropout2d()
        self.conv3 = nn.Conv2d(20, 30, kernel_size=3)
        self.fc1 = nn.Linear(4320, 1024)
        self.fc2 = nn.Linear(1024, 2)
        self.relu = nn.ReLU()
        self.max_pool2d = nn.MaxPool2d(2)
    def forward(self, x):
        x = self.relu(self.max_pool2d(self.conv1(x)))
        x = self.relu(self.max_pool2d(self.conv_drop(self.conv2(x))))
        x = self.relu(self.max_pool2d(self.conv_drop(self.conv3(x))))
        x = x.view(x.shape[0],-1)
        # print(x.shape)
        x = self.relu(self.fc1(x))
        x = self.conv_drop(x)
        x = self.fc2(x)
        return x

In [ ]:
model=Binary_Classifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

# import torch.optim.lr_scheduler.StepLR
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)


In [ ]:
model_node_1 = 'model_3_convs.pt'

model_node_2_fc = 'model_tf_3_convs_retrain_last_fc_node2.pt'
model_node_2_conv = 'model_tf_3_convs_retrain_last_conv_node2.pt'

model_node_3_fc = 'model_tf_3_convs_retrain_last_fc_node3.pt'
model_node_3_conv = 'model_tf_3_convs_retrain_last_conv_node3.pt'

model_node_4_fc = 'model_3_convs_node_4_fc.pt'
model_node_4_conv = 'model_3_convs_node_4_conv.pt'

model_node_5_fc = 'model_3_convs_node5_fc.pt'
model_node_5_conv = 'model_3_convs_node5_conv.pt'

model_node_6_fc = '3convs_node6_fc.pt'
model_node_6_conv = '3convs_node6_conv.pt'

model_node_7_fc = '3convs_node7_fc.pt'
model_node_7_conv = '3convs_node7_conv.pt'

model_node2_node3 = 'modelreuse_node2convmodel_retrainedfc_on_node3.pt'

In [ ]:
if model_node == 1:
  model_path = model_node_1

elif model_node == 2:
  model_path = model_node_2_conv


elif model_node == 3:
  model_path = model_node_3_conv


elif model_node == 4:
  model_path = model_node_4_conv


elif model_node == 5:
  model_path = model_node_5_conv


elif model_node == 6:
  model_path = model_node_6_conv

elif model_node == 7:
  model_path = model_node_7_conv


elif model_node == 23:
  model_path = model_node2_node3

else:
  print("set model_node variable at the top of the file")

In [ ]:
# if os.path.exists(save_path+'/model_3_convs_node6_conv.pt'):
try:
    print('Loading checkpoint: %s' % save_path + model_path)
    path = torch.load(save_path + model_path)
    # epoch = path['epoch']
    model.load_state_dict(path['model'])
    optimizer.load_state_dict(path['optimizer'])
except Exception as e:
  print("nothing happened")
  print(e)

Loading checkpoint: /content/drive/MyDrive/iam_dataset/saved_model/modelreuse_node2convmodel_retrainedfc_on_node3.pt


In [ ]:
from temperature_scaling import ModelWithTemperature
scaled_model = ModelWithTemperature(model)
temp_model = scaled_model.set_temperature(val_dataloader)

Before temperature - NLL: 0.642, ECE: 0.017
Optimal temperature: 1.493
After temperature - NLL: 0.647, ECE: 0.029


In [ ]:
print(temp_model)

ModelWithTemperature(
  (model): Binary_Classifier(
    (conv1): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
    (conv2): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
    (conv_drop): Dropout2d(p=0.5, inplace=False)
    (conv3): Conv2d(20, 30, kernel_size=(3, 3), stride=(1, 1))
    (fc1): Linear(in_features=4320, out_features=1024, bias=True)
    (fc2): Linear(in_features=1024, out_features=2, bias=True)
    (relu): ReLU()
    (max_pool2d): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
)


In [ ]:
result = []
prediction = []
temp_model.eval()
with torch.no_grad():
    for idx, (data, target) in enumerate(test_dataloader):
        output = temp_model(data)
        output_sm = nn.Softmax(dim=1)(output)
        _, predicted = torch.max(output, 1)
        prediction.append(predicted)
        result.append(output_sm)
        
result = torch.stack(result).detach().cpu().numpy()
prediction = torch.stack(prediction).detach().cpu().numpy()

print(result.shape, prediction.shape)

(1213, 16, 2) (1213, 16)


In [ ]:
import pandas as pd
res = pd.DataFrame(result.reshape(-1,2))
pred = pd.DataFrame(prediction.reshape(-1))
y_test_new = y_test[:result.reshape(-1,2).shape[0]]
result_df = pd.DataFrame({'ground truth': y_test_new, 'NN output': prediction.reshape(-1), 'probs 0': res[0], 'probs 1': res[1]})
print(result_df)
# work out the accuracy and other metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
actual = result_df['ground truth']
predicted = result_df['NN output']
# confusion matrix
matrix = confusion_matrix(actual, predicted, labels=[0,1])
print('Confusion matrix : \n',matrix)

# outcome values order in sklearn
tp, fn, fp, tn = confusion_matrix(actual, predicted, labels=[0,1]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy
matrix_2 = classification_report(actual,predicted,labels=[0,1], output_dict=True)
print('Classification report : \n',matrix_2)

matrix_df = pd.DataFrame(matrix_2).transpose()




       ground truth  NN output   probs 0   probs 1
0               0.0          0  0.522608  0.477392
1               0.0          1  0.468868  0.531132
2               0.0          0  0.505892  0.494108
3               0.0          0  0.501319  0.498682
4               0.0          0  0.535124  0.464876
...             ...        ...       ...       ...
19403           0.0          1  0.491070  0.508930
19404           0.0          0  0.543705  0.456295
19405           0.0          0  0.536882  0.463118
19406           0.0          0  0.530265  0.469735
19407           0.0          0  0.534099  0.465901

[19408 rows x 4 columns]
Confusion matrix : 
 [[8487 2325]
 [  26 8570]]
Outcome values : 
 8487 2325 26 8570
Classification report : 
 {'0': {'precision': 0.9969458475273112, 'recall': 0.7849611542730299, 'f1-score': 0.8783441138421733, 'support': 10812}, '1': {'precision': 0.7865993575034419, 'recall': 0.9969753373662168, 'f1-score': 0.8793802267713303, 'support': 8596}, 'accuracy':

In [ ]:
def save_results(model_node):
    result_df.to_csv(save_path + "calibrated_softmax/result_probs_3conv_"+ "node"+ str(model_node)+ "conv" + "_calibrated.csv")
    matrix_df.to_csv(save_path + "calibrated_softmax/classification_report_3conv" + "node"+str(model_node)+ "conv" +".csv", header=True, index=True)
    print("model " + str(model_node) + " results saved!")

In [ ]:
if model_node == 1:
  save_results(1)
elif model_node == 2:
  save_results(2)
elif model_node == 3:
  save_results(3)
elif model_node == 4:
  save_results(4)
elif model_node == 5:
  save_results(5)
elif model_node == 6:
  save_results(6)
elif model_node == 7:
  save_results(7)
elif model_node == 23:
  result_df.to_csv(save_path + "calibrated_softmax/result_probs_3conv_modelreuse_node23model_on_node1_calibrated.csv")
  matrix_df.to_csv(save_path + "calibrated_softmax/classification_report_3conv_modelreuse_node23model_on_node1.csv", header=True, index=True)


